In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9 

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.stat import Correlation
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .master("local[4]") \
    .config("spark.driver.maxResultSize", "8g") \
    .getOrCreate()

In [3]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df = spark.read.parquet('../../data/cleanedDataset_parquet')

In [3]:
df.show(5)

+--------------------+----------------+-----------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+--------------------+---------------------+--------------------+---------------------+------------------+
|            id_track|popularity_track|duration_ms|              genres|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|sum_artist_followers|sum_artist_popularity|avg_artist_followers|avg_artist_popularity|               age|
+--------------------+----------------+-----------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+--------------------+---------------------+--------------------+---------------------+------------------+
|00AeAaSNbe92PRrst...|               3|     156067|[classic czech p

In [4]:
df.printSchema()

root
 |-- id_track: string (nullable = true)
 |-- popularity_track: integer (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- release_date: date (nullable = true)
 |-- danceability: float (nullable = true)
 |-- energy: float (nullable = true)
 |-- key: integer (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode: integer (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- acousticness: float (nullable = true)
 |-- instrumentalness: float (nullable = true)
 |-- liveness: float (nullable = true)
 |-- valence: float (nullable = true)
 |-- tempo: float (nullable = true)
 |-- time_signature: integer (nullable = true)
 |-- sum_artist_followers: long (nullable = true)
 |-- sum_artist_popularity: long (nullable = true)
 |-- avg_artist_followers: double (nullable = true)
 |-- avg_artist_popularity: double (nullable = true)
 |-- age: double (nullable = true)



In [5]:
from pyspark.ml.feature import QuantileDiscretizer 

qds = QuantileDiscretizer(relativeError=0.0001, handleInvalid="error", numBuckets=10, inputCol="popularity_track", outputCol="label")

df = qds.setHandleInvalid("keep").fit(df).transform(df)

In [6]:
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf, StringType
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [7]:
assembler = VectorAssembler( 
inputCols=['age',
 'duration_ms',
 'danceability',
 'energy',
 'key',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature',
 'sum_artist_followers',
 'sum_artist_popularity'], 
outputCol="features")
output=assembler.transform(df)

In [8]:
final_data = output.select( "features", "label")

In [9]:
from pyspark.ml.feature import StandardScaler


scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(final_data)

# Normalize each feature to have unit standard deviation.
df = scalerModel.transform(final_data)

In [25]:
df = df.select("scaledFeatures", "label")

In [26]:
train, test = df.randomSplit([0.7, 0.3], seed = 10)

In [27]:
layers = [10, 4, 2, 10]

In [28]:
mlp = MultilayerPerceptronClassifier(labelCol='label',
                                            featuresCol='scaledFeatures',
                                            maxIter=100,
                                            layers=layers,
                                            blockSize=128,
                                            seed=1234)

mlpModel = mlp.fit(train)


predictions = mlpModel.transform(test)

In [29]:
predictions.select("scaledFeatures", "prediction").show()

Py4JJavaError: An error occurred while calling o551.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 283.0 failed 1 times, most recent failure: Lost task 0.0 in stage 283.0 (TID 1248) (192.168.56.1 executor driver): org.apache.spark.SparkException: Failed to execute user defined function(ProbabilisticClassificationModel$$Lambda$3591/196168134: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:755)
	a

In [31]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

Py4JJavaError: An error occurred while calling o555.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 285.0 failed 1 times, most recent failure: Lost task 3.0 in stage 285.0 (TID 1253) (192.168.56.1 executor driver): org.apache.spark.SparkException: Failed to execute user defined function(ProbabilisticClassificationModel$$Lambda$3591/196168134: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:755)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	